# 본문 스크래핑

In [1]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
# import chromedriver_auto


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('headless') # 창 숨기기


In [2]:
# from user_agent import generate_user_agent, generate_navigator

# print(generate_user_agent(device_type='desktop'))
# print(generate_user_agent(os='win', device_type='desktop'))

# navigator = generate_navigator()

# random_user = generate_user_agent(os='win', device_type='desktop')
# print(random_user)

In [3]:
from selenium.webdriver.chrome.options import Options

# chrome_options = Options()
# chrome_options.add_argument(random_user)
driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = options)

driver.maximize_window()
driver.get("https://www.naver.com/")
driver.implicitly_wait(10)


In [4]:
url_df = pd.read_csv("./나무문화/2022/나무문화_링크_2022_8월.csv", encoding='UTF-8')
print(url_df.shape, url_df.columns)
url_df


(20814, 3) Index(['url', 'title', 'date'], dtype='object')


,url,title,date
0,https://blog.naver.com/smileseosin/222863511415,불편한 편의점2 김호연 작가 특강이 송천도서관 문화가 있는 날...,2022.08.31.
1,https://blog.naver.com/bluex/222863510213,추석 그림책 명절 독서 활동 달님에게 소원 편지쓰기 색칠하기,2022.08.31.
2,https://blog.naver.com/cmdsc809/222863508877,"[미국 여행] 0419-0502세 달 살이 마무리, 사랑하는 친구들...",2022.08.31.
3,https://blog.naver.com/san1917/222863508568,[오스트리아] 멜크수도원,2022.08.31.
4,https://blog.naver.com/hnmh2332/222863508119,추석연휴 놀러 갈만한곳 강원도 여행 평창 청옥산 육백마지기,2022.08.31.
...,...,...,...
20809,https://blog.naver.com/show656/222835904038,내가 찾던 그 맛 바로 여기 라멘맛집에서 발견 : 광주 동명동...,2022.08.01.
20810,https://blog.naver.com/sonia30/222835904165,강릉 여행::오죽헌 입장료 및 주차장 (feat.시립박물관),2022.08.01.
20811,https://blog.naver.com/october2nd_/222835901895,[대전궁동맛집] 회쳐모여 | 대전유성횟집 추천 | 가성비좋은...,2022.08.01.
20812,https://blog.naver.com/moviemagazine/222835583839,[스포영화소개] 앎에서 오는 공포 <미드소마>,2022.08.01.


In [5]:
blog_links = url_df['url'].to_list()
print(len(blog_links))


20814


In [6]:
# df = pd.DataFrame(columns = ['url','본문'])
# df.to_csv("./나무문화/2022/나무문화_본문_2022_8월.csv", index=False, encoding='UTF-8')
# df.to_excel("./나무문화/2022/나무문화_본문_2022_8월.xlsx", index=False, encoding='UTF-8')

In [7]:
df = pd.read_csv("./나무문화/2022/나무문화_본문_2022_8월.csv", encoding='UTF-8')
df

,url,본문
0,https://blog.naver.com/smileseosin/222863511415,50m\n© NAVER Corp.\n송천도서관\n전라북도 전주시 덕진구 솔내로 21...
1,https://blog.naver.com/bluex/222863510213,즐거운 책 놀이터\n독서교육 강사 이숙영입니다.\n어느 새 무더운 여름이 지나고\n...
2,https://blog.naver.com/cmdsc809/222863508877,"다시 짧은 시간이 지났고,\n바람이 불었고,\n그렇게 앉아있는 동안\n나는 내가 이..."
3,https://blog.naver.com/san1917/222863508568,멜크수도원\n멜크수도원\n2014년 4월 22일~2014년 5월2일\n베네룩스 3국...
4,https://blog.naver.com/hnmh2332/222863508119,추석연휴 놀러 갈만한곳 강원도 여행 평창 청옥산 육백마지기\n글&사진 : 초록바다 ...
...,...,...
10003,https://blog.naver.com/harb/222850258072,블로그\n운동해 257개의 글\n운동해\n목록열기\n걷기 시작 운동해 \n2022...
10004,https://blog.naver.com/jyunjyuni/222850256657,"제주도에 왔는데 맛있는걸 다 먹어보고 가지 않으면 너무 아쉬울 것 같아, 우리는 아..."
10005,https://blog.naver.com/yangaram1/222850257170,2022. 8. 3. 아산병원 장례식장에 문상을 다녀왔습니다. 잠실나루역에서 장례식...
10006,https://blog.naver.com/kusu6888/222850255842,"안녕하세요, 제 68회 백제문화제 서포터즈 ""알밤막걸리""입니다.\n오늘 알려드릴 것..."


In [8]:
len(df)

10008

In [9]:
#url_df.iloc[11265]

In [10]:
df.isnull().sum()

url    0
본문     0
dtype: int64

In [11]:
# 이건 필요할때 쓰면 됨. 블로그가 닫혔거나 삭제되었을때 빈 본문을 가진 데이터프레임 추가.
# content_dic = {'url' : ["https://blog.naver.com/silhouette_04/222060553502"], '본문' : [None]}
# additional_df = pd.DataFrame(content_dic)
# df = df.append(additional_df, ignore_index = True)

In [14]:
#### 삭제/비공개 링크 인덱스도 매번 바꿔줄 것!!! ###
# contents = []
# old_blogs = []
hide_link = []
waitsec = random.uniform(4,4.5)

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

for i in tqdm(blog_links[len(df):]):
    content_dic = {'url' : [], '본문' : []}
    content_dic['url'].append(i)
    driver.get(i)
    driver.implicitly_wait(10)
    time.sleep(waitsec)
    try:
        driver.switch_to.frame("mainFrame")
    except TimeoutException as e:
        print('시간 초과 오류 : ', e)
        content_dic['본문'].append('수집불가능')
    except UnexpectedAlertPresentException as e:
    ### 이 오류 해결함. 에러 코드 파일에서 직접 임포트하면 된다!
        print(e)
                                                                            # 여기도 바꿔야 함
        print('{}번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.'.format(blog_links.index(i)))
        hide_link.append(i)
        time.sleep(1.5)
        driver.switch_to.frame('mainFrame')
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        content_dic['본문'].append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        content_dic['본문'].append(a)
        #old_blogs.append(i)
    additional_df = pd.DataFrame(content_dic)
    df = pd.concat([df,additional_df])
    df.reset_index(drop=True, inplace=True)
    #df.to_excel("./나무문화/2020/나무문화_본문_2020_8월.xlsx", index=False, encoding='UTF-8')
    print(blog_links.index(i), df['본문'].tolist().index(a))
    
    display.clear_output(wait=True) # 출력지우기
    # if blog_links.index(i) != df['본문'].tolist().index(a):
    #     break


KeyboardInterrupt: 

In [15]:
df

,url,본문
0,https://blog.naver.com/smileseosin/222863511415,50m\n© NAVER Corp.\n송천도서관\n전라북도 전주시 덕진구 솔내로 21...
1,https://blog.naver.com/bluex/222863510213,즐거운 책 놀이터\n독서교육 강사 이숙영입니다.\n어느 새 무더운 여름이 지나고\n...
2,https://blog.naver.com/cmdsc809/222863508877,"다시 짧은 시간이 지났고,\n바람이 불었고,\n그렇게 앉아있는 동안\n나는 내가 이..."
3,https://blog.naver.com/san1917/222863508568,멜크수도원\n멜크수도원\n2014년 4월 22일~2014년 5월2일\n베네룩스 3국...
4,https://blog.naver.com/hnmh2332/222863508119,추석연휴 놀러 갈만한곳 강원도 여행 평창 청옥산 육백마지기\n글&사진 : 초록바다 ...
...,...,...
10464,https://blog.naver.com/binbin803803/222849654041,전라도 보성으로 보성CC 1박2일골프 라운딩\n안녕하세요. 오늘은 전라도 보성의 골...
10465,https://blog.naver.com/ljb200350/222840503784,"- 더 발전하여 새롭게 시작하는 The ROUTE', 이번에 다룰 노선은 김포 90..."
10466,https://blog.naver.com/hinesiodi34/222849650661,블로그\n게시판 52개의 글\n게시판\n목록열기\n게시판\n밤의 좋은 영양성분들과 ...
10467,https://blog.naver.com/zerg567890/222849647869,"© sommi, 출처 Unsplash\n0. 랩틸리언 반론글에서 신화로\n최근에 지..."


In [16]:
df.to_csv("./나무문화/2022/나무문화_본문_2022_8월.csv", index=False, encoding='UTF-8')